# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory


## Dataset

### Overview
The dataset being used is the housing dataset for California based on the 1990 consensus. This has been retrieved form kaggle and has 10 columns all used to predict medain house value.

## Setting up Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = "\n")

quick-starts-ws-135901
aml-quickstarts-135901
southcentralus
a24a24d5-8d87-4c8a-99b6-91ed2d2df51f


## Initialising Experiment

In [21]:
# choose a name for experiment
experiment_name = 'titanic_automl'
project_folder = './titanic_automl'
os.makedirs(project_folder, exist_ok = True)
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
titanic_automl,quick-starts-ws-135901,Link to Azure Machine Learning studio,Link to Documentation


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

## Creating or checking for existing compute cluster

In [22]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aml_compute_name = 'computecluster1' 
try:
    compute_target = ComputeTarget(workspace = ws, name = aml_compute_name)
    print("Existing cluster. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", max_nodes = 4)
    compute_target = ComputeTarget.create(ws,aml_compute_name, compute_config)
compute_target.wait_for_completion(show_output = True)

Existing cluster. Use it.

Running


## Preparing Data

In [24]:
import pandas as pd
data = pd.read_csv('titanic_dataset.csv', header = 0)

In [25]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [7]:
from sklearn.model_selection import train_test_split

In [28]:
def Impute_missing_values(data):
    data.drop(columns=['Cabin'],inplace=True)
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Embarked'].fillna('S', inplace=True)
    data['Fare'].fillna(data['Fare'].median()) 
    return data

def Family_type(number):
    if number==0:
        return 'Alone'
    elif number>0 and number<=4:
        return 'Medium'
    else:
        return 'Large'

def Transform_data(data):
    data['Family_size']=data['Parch']+data['SibSp']
    data['Family_type']=data['Family_size'].apply(Family_type)
    data.drop(columns=['SibSp', 'Parch', 'Family_size'], inplace=True)
    data.loc[ data['Age'] <= 16, 'Age'] = 1
    data.loc[(data['Age'] > 16) & (data['Age'] <= 26), 'Age'] = 2
    data.loc[(data['Age'] > 26) & (data['Age'] <= 36), 'Age'] = 3
    data.loc[(data['Age'] > 36) & (data['Age'] <= 62), 'Age'] = 4
    data.loc[ data['Age'] > 62, 'Age'] = 5
    data.loc[data['Fare'] <= 17, 'Fare'] = 1,
    data.loc[(data['Fare'] > 17) & (data['Fare'] <= 30), 'Fare'] = 2,
    data.loc[(data['Fare'] > 30) & (data['Fare'] <= 100), 'Fare'] = 3,
    data.loc[ data['Fare'] > 100, 'Fare'] = 4
    return data

def clean_data(data):
    data = Impute_missing_values(data)
    data.head()
    x_data = Transform_data(data)
    x_data=pd.get_dummies(data=x_data, columns=['Age' ,'Fare',  'Pclass', 'Sex', 'Embarked', 'Family_type'], drop_first=True)
    x_data.drop(columns=['Ticket', 'PassengerId', 'Name','Age_5.0'],inplace=True)
    y_data = x_data.pop("Survived")
    return x_data, y_data

x, y = clean_data(data)
x.head()

#Split the dataset into train and test dataset. Combine x_train and y_train. 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
data_train = pd.concat([x_train,y_train], axis=1)
data_test = pd.concat([x_test,y_test], axis=1)

#Convert x_train and y_train (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path_train = 'data/train.csv'
path_test = 'data/test.csv'
data_train.to_csv(path_train)
data_test.to_csv(path_test)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/test.csv'))])
print("Successfully converted the dataset to TabularDataset format.")


Uploading an estimated of 2 files
Uploading data/test.csv
Uploaded data/test.csv, 1 files out of an estimated total of 2
Uploading data/train.csv
Uploaded data/train.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Successfully converted the dataset to TabularDataset format.


In [31]:
from azureml.train.automl import AutoMLConfig

#Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="Survived",
    n_cross_validations=5,
    compute_target=compute_target)

In [12]:
test = TabularDatasetFactory.from_delimited_files([(datastore, 'housing_test.csv')])

## AutoML Configurations

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [32]:
from azureml.widgets import RunDetails
automl_run = experiment.submit(automl_config, show_output = True)
RunDetails(automl_run).show()

Running on remote.
No run_configuration provided, running on computecluster1 with default configuration
Running on remote compute: computecluster1
Parent Run ID: AutoML_ad43f349-8d0c-4cb6-9739-4f4ac96a8bed

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing v

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [33]:
automl_run.wait_for_completion()

{'runId': 'AutoML_ad43f349-8d0c-4cb6-9739-4f4ac96a8bed',
 'target': 'computecluster1',
 'status': 'Completed',
 'startTimeUtc': '2021-01-24T17:00:06.386942Z',
 'endTimeUtc': '2021-01-24T17:37:52.613463Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'computecluster1',
  'AMLSettingsJsonString': '{"path":null,"name":"titanic_automl","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-135901","workspace_name":"quick-starts-ws-135901","region":"southcentralus","compute_target":"computecluster1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_cla

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [35]:
best_run_AutoML, best_model_AutoML = automl_run.get_output()
best_run_metrics_AutoML = best_run_AutoML.get_metrics()
print("Best run Id: ",best_run_AutoML.id)
print("Accuracy: ", best_run_metrics_AutoML['accuracy'])
print("Other details: ")
print("Fitted model:",best_model_AutoML)

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Best run Id:  AutoML_ad43f349-8d0c-4cb6-9739-4f4ac96a8bed_28
Accuracy:  0.8090121146459174
Other details: 
Fitted model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                             KNeighborsClassifier(algorithm='auto',
                                                                                                  leaf_size=30,
                                           

In [38]:
import joblib
from azureml.core.model import Model

#Save the best model
os.makedirs('results', exist_ok=True)
joblib.dump(best_model_AutoML, filename="results/automl_model.pkl")
model = automl_run.register_model(model_name=best_run_AutoML.properties['model_name'], description='Best AutoML model')
print("Model saved successfully")

Model saved successfully


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service